[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Machine Learning Methods

## Introduction to Optimization - The Gradient Descent Algorithm

> Notebook by:
> - Silvian Grigoriu RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 18/01/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/silviang10/silviang10/blob/master/0003GradientDescent.ipynb)

In [1]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Miscellaneous
import datetime
import os
from platform import python_version
import random
import warnings
import yaml

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show
import plotly.graph_objects as go
import plotly.io as pio

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, interactive, IntSlider, Layout, VBox

In [2]:
# Configuration
%matplotlib inline

warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [3]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [4]:
# Fixel Algorithms Packages


In [5]:
# Parameters




## The Function

* Consider the function $f$:

$$ f \left( {x}_{1}, {x}_{2} \right) = {x}_{1}^{2} + 10 {x}_{2}^{2} $$

* The gradient of $f$:

$$ \nabla f \left( {x}_{1}, {x}_{2} \right) = \begin{bmatrix} 2 {x}_{1} \\ 20 {x}_{2} \end{bmatrix} $$


In [6]:
# Define the Function

def f(mX: np.ndarray):
    
    vX1 = mX[0, :]
    vX2 = mX[1, :]
    
    return (vX1 ** 2) + (10 * (vX2 ** 2))


In [7]:
# Define the Gradient

def GradF(vX):

    x1     = vX[0]
    x2     = vX[1]
    vGradF = np.array([2 * x1, 20 * x2])
    
    return vGradF

## The Gradient Descent Algorithm

The algorithm is defined iteratively as:

$$ \boldsymbol{x}_{k+1} = \boldsymbol{x}_{k} - \mu \nabla f \left( \boldsymbol{x}_{k} \right) $$

In [8]:
# The Gradient Descent 
vX0 = np.array([-9.8, 1.7]) #<! Initial point
K   = 50                    #<! Number of iterations (Keep 50 for tne interactive part)
µ   = 0.09                  #<! Step size

vX      = vX0.copy()
mX      = np.full((2, K), np.nan) #<! The result per iteration

mX[:, 0] = vX0
for kk in range(1, K):
    vX          = vX - µ * GradF(vX)
    mX[:, kk]   = vX

In [9]:
# Plotting the Path

# Grid and Surface
v1         = np.linspace(-10, 10, 51)
v2         = np.linspace(-2, 2, 51)
mXX1, mXX2 = np.meshgrid(v1, v2)
mXX        = np.c_[mXX1.ravel(), mXX2.ravel()].T
mY         = f(mXX)
mY.shape   = mXX1.shape

# Plotting  (PlotLy)
contours = dict(z = dict(show = True, start = -1, end = 100, size = 10, color = 'black'))
marker   = dict(size = 5, color = list(range(K + 1)), colorscale = ["magenta", "yellow"])
line     = dict(color = 'black', width = 2)
# title    = dict(text = r'$\mu =$' + str(µ), y = 0.7, x = 0.5, xanchor = 'center', yanchor = 'top')
title    = dict(text = r'μ = ' + str(μ), y = 0.7, x = 0.5, xanchor = 'center', yanchor = 'top')
font     = dict(size = 16)
camera   = dict(eye = dict(x = 0, y = -1, z = 4))
scene    = dict(xaxis = dict(title = 'x1', range = [-10, 10]),
                yaxis = dict(title = 'x2', range = [-2, 2]),
                zaxis = dict(title = 'f',  range = [-1/2, 200]))

Surf1  = go.Surface(x = mXX1, y = mXX2, z = mY, colorscale = 'Viridis', opacity = 0.95, showscale = False, contours = contours)
Scat1  = go.Scatter3d(x = mX[0, :], y = mX[1, :], z = f(mX), marker = marker, line = line)

layout = go.Layout(width = 1000, height = 500, margin = dict(r = 0, l = 0, b = 0, t = 0),
                   title = title, font = font, scene = scene, scene_camera = camera,
                   scene_aspectmode = 'manual', scene_aspectratio = dict(x = 5, y = 1, z = 1))

fig   = go.Figure(data = [Surf1, Scat1], layout = layout)
fig.show()

In [10]:
# Interactive Visualization

def PlotGradientDescent(figWidget, µ, K = 15):
    vX      = vX0.copy()
    mX      = np.full((2, K), np.nan)
    mX[:,0] = vX0
    for kk in range(1, K):
        vX       = vX - µ * GradF(vX)
        mX[:,kk] = vX

    with figWidget.batch_update():
        figWidget.data[1].x = mX[0, :]
        figWidget.data[1].y = mX[1, :]
        figWidget.data[1].z = f(mX)
        figWidget.layout.title.text = f'µ = {µ}'

In [11]:
# PlotLy Widget
figWid = go.FigureWidget(data = fig.data, layout = fig.layout)

In [12]:
# Sliders
KSlider = IntSlider(min = 1, max = 50, step = 1, value = 15, layout = Layout(width = '30%'))
µSlider = FloatSlider(min = 0, max = .15, step = .01, value = .01, layout = Layout(width = '30%'))

In [13]:
# Interactive Widget (Won't work on Google Colab)
hPlotGradientDescent = lambda µ, K: PlotGradientDescent(figWid, µ, K)
vertBox = VBox((figWid, interactive(hPlotGradientDescent, µ = µSlider, K = KSlider)))
vertBox

    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
…